In [226]:
import json
import glob
import csv
import bs4
import re
import os
import subprocess
from openpyxl import load_workbook
import pandas as pd
import pandas_datareader as pdr
import numpy as np
from selenium import webdriver
import yfinance as yf
import time
import datetime
from IPython.display import display

In [227]:
DFList:list=[pd.DataFrame] * 5;
#perform arithmetic operations on date
#print(datetime.datetime.today().strftime('%d-%m-%Y')-datetime.timedelta(days=1));
print(type(datetime.datetime.today()-datetime.timedelta(days=2)));
DateConvert_Str:datetime.datetime=datetime.datetime.today()-datetime.timedelta(days=1);
DateString:str=datetime.datetime.strftime(DateConvert_Str,'%d-%m-%Y');
ThaiSS_FileStr:str='/Users/cedricyang/Downloads/Short Sales_'+DateString+' to '+DateString+'.xlsx';
#or implement checks to see if the files was downloaded successfully
DirListFile:list=os.listdir('/Users/cedricyang/Downloads/');
List_of_file:list=glob.glob('/Users/cedricyang/Downloads/*.xlsx');
NameString:str='/Users/cedricyang/Downloads/';

<class 'datetime.datetime'>


In [228]:
FileIO=open('IRSG.json','r');
DictObjs:dict=json.load(FileIO);
DFList[0]:pd.DataFrame=pd.json_normalize(DictObjs,record_path=['result','records']);
Sora_Quantile=DFList[0]['comp_sora_3m'].quantile([0.25,0.5,0.75,0.95,1.0]).to_dict();
DFList[0]['Indicator 3M > 1M']=DFList[0]['comp_sora_3m']>DFList[0]['comp_sora_1m'];
DFList[0]['Indicator 3M > 6M']=DFList[0]['comp_sora_3m']>DFList[0]['comp_sora_6m'];
display(DFList[0])

,end_of_day,comp_sora_1m,comp_sora_3m,comp_sora_6m,aggregate_volume,timestamp,Indicator 3M > 1M,Indicator 3M > 6M
0,2023-04-24,NaN,NaN,NaN,NaN,1682339863,False,False
1,2023-04-21,3.5037,3.5969,3.3457,2476.0,1682339863,True,True
2,2023-04-20,3.4861,3.5802,3.3306,2728.0,1682339863,True,True
3,2023-04-19,3.4985,3.5816,3.3290,1901.0,1682339863,True,True
4,2023-04-18,3.5178,3.5851,3.3253,1790.0,1682339863,True,True
...,...,...,...,...,...,...,...,...
495,2021-05-04,0.1349,0.1884,0.1782,1288.0,1682339863,True,True
496,2021-05-03,0.1328,0.1875,0.1778,1318.0,1682339863,True,True
497,2021-04-30,0.1294,0.1862,0.1772,1230.0,1682339863,True,True
498,2021-04-29,0.1202,0.1850,0.1779,1782.0,1682339863,True,True


In [51]:
Custom_Options=webdriver.ChromeOptions();
Custom_Options.add_argument("--lang=en");
prefs = {
  "translate_whitelists":{"th":"en"},
  "translate":{"enabled":"true"}
};
print(prefs.keys());
Custom_Options.add_experimental_option("prefs", prefs);
ThaiSetTrade:webdriver.chrome.webdriver.WebDriver=webdriver.Chrome(options=Custom_Options);
ThaiSetTrade.get('https://www.settrade.com/th/home');
#//*[@id="modalFloatingAds"]/div/div[1]/button/svg
ThaiSetTrade.find_element(webdriver.common.by.By.XPATH,'//*[@id="modalFloatingAds"]/div/div[1]/button').click();

dict_keys(['translate_whitelists', 'translate'])


In [229]:
#use firefox to avoid issue while code is running and im surfing the net
DriverObjs:webdriver.firefox.webdriver.WebDriver=webdriver.Firefox();
DriverObjs.get('https://www.set.or.th/en/market/statistics/short-sell');
time.sleep(2);
print(DriverObjs.title);
DriverObjs.find_element(webdriver.common.by.By.XPATH,'//*[@id=\"__layout\"]/div/div[2]/div[3]/div/div/div[2]/div/div[2]/button').click();
pass; #similar to __asm__("nop")
time.sleep(2);
DriverObjs.close();

Short Selling - The Stock Exchange of Thailand


In [232]:
FileDate:datetime.date=datetime.date.today()-datetime.timedelta(3);
DateConcate:str=datetime.datetime.strftime(FileDate,'%d-%m-%Y');
BackupFileStr:str='/Users/cedricyang/Downloads/Short\ Sales_'+DateConcate+'\ to\ '+DateConcate+'.xlsx';
print(BackupFileStr);
TerminalCmd='open -a /Applications/Microsoft\ Excel.app ' + BackupFileStr;
subprocess.call('open -a /Applications/Microsoft\ Excel.app '+BackupFileStr,shell=True);

/Users/cedricyang/Downloads/Short\ Sales_21-04-2023\ to\ 21-04-2023.xlsx


In [233]:
LoopList:list=glob.glob('/Users/cedricyang/Downloads/*.xlsx');
Regex:re.Pattern=re.compile('/Users/cedricyang/Downloads/Short Sales_'+DateConcate+'?');
matches:list=[];
matches=[stringval for stringval in LoopList if re.match(Regex,stringval)];
print(matches);
print(os.curdir)
#print(os.system('ls -lt | head -5'));

['/Users/cedricyang/Downloads/Short Sales_21-04-2023 to 21-04-2023.xlsx']
.


In [41]:
ThaiSS_DF:pd.DataFrame=pd.read_excel(io=matches[0],engine='openpyxl',header=5,index_col=0);
ThaiSS_DF=ThaiSS_DF[:200];
ThaiSS_DF=ThaiSS_DF.sort_values(by=['Turnover (Baht)','Symbol'],ascending=False)
ThaiSS_DF=ThaiSS_DF.reset_index();
ThaiSS_DF['USD Eqiv Amt']=(ThaiSS_DF['Turnover (Baht)'] / 34.12);
display(ThaiSS_DF[:30])

,Symbol,Volume (Shares),Turnover (Baht),% Short Sale Volume Comparing with Auto Matching,USD Eqiv Amt
0,BDMS,9615300.0,284699500.0,0.097836,8.344065e+06
1,CPALL,3910300.0,248972500.0,0.088697,7.296967e+06
2,KBANK,1574900.0,208057350.0,0.137492,6.097812e+06
3,BANPU,19692600.0,188380545.0,0.155531,5.521118e+06
4,EA-R,1958400.0,147195425.0,0.109850,4.314051e+06
5,CBG-R,1720100.0,143862150.0,0.153090,4.216358e+06
6,AOT,1855700.0,134853375.0,0.068424,3.952326e+06
7,GULF,2064700.0,110255900.0,0.125224,3.231416e+06
8,ADVANC,455000.0,96662100.0,0.095777,2.833004e+06
9,JMT,2140700.0,86425775.0,0.064566,2.532995e+06


In [234]:
#creating a list of list
ListLoop:list=[list]*650;
print(ListLoop);

[<class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 

In [235]:
HtmlIO_File=open('HK_X_SS.html');
soupObj=bs4.BeautifulSoup(HtmlIO_File,'html.parser');
preTags:str=soupObj.select_one('pre').text;
SplitList:list=preTags.split('\n')[5:];
SplitList_len:int=len(SplitList);

In [236]:
jjj:int=0;
List_indexer:int=0
for i in range(2,SplitList_len):
    ListLoop[List_indexer]=SplitList[i].split(" ");
    ListLoop[List_indexer]=[x for x in ListLoop[List_indexer] if x != '']
    List_indexer=List_indexer+1;
    #x for x in list_with_empty_strings if x != ''

LoopCounters:int=0;
OuterList_len:int=len(ListLoop);
#Testing Relational Operator

while (bool(re.match(r'%82[0-9]',str(ListLoop[LoopCounters][0]))) is not True):
    Each_List_len=len(ListLoop[LoopCounters]);
    if (ListLoop[LoopCounters][0]=='%'): #indicates it's a STOCK traded in NON-HKD
        ListLoop[LoopCounters][Each_List_len-Each_List_len]='NON-HKD Stock/ETF';
        ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
        ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''))
        LoopCounters=LoopCounters+1;
        continue;
    elif (ListLoop[LoopCounters][0]=='*'):
        LoopCounters=LoopCounters+1;
        continue;
    #Stored the converted value back to memory.. slow CPU operations
    ListLoop[LoopCounters][Each_List_len-Each_List_len]=int(ListLoop[LoopCounters][Each_List_len-Each_List_len]);
    ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
    ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''));
    LoopCounters=LoopCounters+1;
    continue;
print(ListLoop);

[[1, 'CKH', 'HOLDINGS', 472500, 24245225], [2, 'CLP', 'HOLDINGS', 153000, 8820375], [3, 'HK', '&', 'CHINA', 'GAS', 696000, 4760950], [4, 'WHARF', 'HOLDINGS', 38000, 675220], [5, 'HSBC', 'HOLDINGS', 734000, 40770440], [6, 'POWER', 'ASSETS', 32500, 1414300], [8, 'PCCW', 57000, 230660], [10, 'HANG', 'LUNG', 'GROUP', 15000, 205420], [11, 'HANG', 'SENG', 'BANK', 30800, 3520300], [12, 'HENDERSON', 'LAND', 431000, 11410900], [13, 'HUTCHMED', 151000, 4031400], [14, 'HYSAN', 'DEV', 8000, 183850], [16, 'SHK', 'PPT', 52500, 5614000], [17, 'NEW', 'WORLD', 'DEV', 120000, 2413600], [19, 'SWIRE', 'PACIFIC', 'A', 51000, 2973000], [20, 'SENSETIME-W', 41559000, 102722660], [23, 'BANK', 'OF', 'E', 'ASIA', 28800, 293020], [27, 'GALAXY', 'ENT', 943000, 53940300], [35, 'FE', 'CONSORT', 'INTL', 8000, 15880], [38, 'FIRST', 'TRACTOR', 6000, 24100], [41, 'GREAT', 'EAGLE', 'H', 3000, 47060], [66, 'MTR', 'CORPORATION', 74500, 2873325], [69, 'SHANGRI-LA', 'ASIA', 2000, 14780], [81, 'CH', 'OVS', 'G', 'OCEANS', 1819

In [237]:
#using "pop" function to retain useful data
def ColLength4(ArgList:list):
    print("Calling 4");
    ArgList.pop(1);
    print(ArgList)
    return;

def ColLength5(ArgList:list):
    print("Calling 5");
    ArgList.pop(1);
    ArgList.pop(1);
    return;

def ColLength6(ArgList:list):
    print("Calling 6");
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    return;

def ColLength7(ArgList:list):
    print("Calling 7");
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    ArgList.pop(1);
    return;

#creating a dictionary of Function Pointers
CreateFuncPtr:dict=dict.fromkeys([4,5,6,7]);
print(CreateFuncPtr);
CreateFuncPtr[4]=ColLength4;
CreateFuncPtr[5]=ColLength5;
CreateFuncPtr[6]=ColLength6;
CreateFuncPtr[7]=ColLength7;

for Length_Iteration in ListLoop:
    if (Length_Iteration[0]=='NON-HKD Stock/ETF'):
        continue; #for simpliciy skip NON-HKD stocks
    elif (re.match(r'%82[0-9]',str(Length_Iteration[0]))):
        break;
    Find_Len:int=len(Length_Iteration);
    CreateFuncPtr[Find_Len](Length_Iteration);

print(ListLoop);

{4: None, 5: None, 6: None, 7: None}
Calling 5
Calling 5
Calling 7
Calling 5
Calling 5
Calling 5
Calling 4
[8, 57000, 230660]
Calling 6
Calling 6
Calling 5
Calling 4
[13, 151000, 4031400]
Calling 5
Calling 5
Calling 6
Calling 6
Calling 4
[20, 41559000, 102722660]
Calling 7
Calling 5
Calling 6
Calling 5
Calling 6
Calling 5
Calling 5
Calling 7
Calling 5
Calling 6
Calling 6
Calling 6
Calling 5
Calling 5
Calling 5
Calling 5
Calling 6
Calling 5
Calling 6
Calling 5
Calling 6
Calling 5
Calling 6
Calling 5
Calling 5
Calling 6
Calling 5
Calling 5
Calling 6
Calling 5
Calling 5
Calling 6
Calling 5
Calling 4
[267, 329000, 3179440]
Calling 5
Calling 5
Calling 6
Calling 5
Calling 4
[286, 66000, 34320]
Calling 5
Calling 6
Calling 6
Calling 4
[297, 280000, 271620]
Calling 4
[302, 6000, 14700]
Calling 5
Calling 6
Calling 5
Calling 4
[316, 20000, 3292600]
Calling 4
[317, 30000, 320440]
Calling 4
[322, 204000, 2739440]
Calling 5
Calling 5
Calling 5
Calling 5
Calling 5
Calling 5
Calling 5
Calling 5
Callin

In [238]:
#create a daily.csv file
CSV_DateStr:datetime.date=datetime.date.today();
Date_Str:str=datetime.datetime.strftime(CSV_DateStr,'%Y%m%d');
DailyFileName:str='HK_Short_Sell_Snapshot_'+Date_Str+'.csv';
#print(DailyFileName);
os.system('touch '+DailyFileName);
print(os.listdir());

['HK_Short_Sell_Snapshot_20230424.csv', 'HK_Short_Sell_Snapshot_20230418.csv', 'ThaiSetInfo.ipynb', 'HK_Short_Sell_Snapshot_20230419.csv', 'HKss.json', '.DS_Store', 'HK_Short_Sell_Snapshot_20230421.csv', 'HK_Short_Sell_Snapshot_20230420.csv', 'TestFile.csv', 'geckodriver.log', 'ThaiWarrants.csv', 'Thai_WarrantInfo.csv', 'IRSG.json', '.ipynb_checkpoints', 'HK_X_SS.html', 'KoreaX_analytics.ipynb', '.git', 'QueryCmd.sh', 'FedNY.json', 'ListofList.csv', 'ReferenceRate.json', 'SecuritiesLending.ipynb']


In [239]:
with open(DailyFileName, "w", newline="\n") as FileIOs:
    writer = csv.writer(FileIOs,delimiter=',',quoting=csv.QUOTE_MINIMAL);
    writer.writerow(['Tickers','#Number of shs','HKD Notional']);
    #append a header to the CSV file
    for rows in ListLoop:
        if (re.match(r'%82[0-9]',str(rows[0]))):
            break;
        print(rows); #for easy debugging.. remove print for to speed up
        writer.writerow(rows);

[1, 472500, 24245225]
[2, 153000, 8820375]
[3, 696000, 4760950]
[4, 38000, 675220]
[5, 734000, 40770440]
[6, 32500, 1414300]
[8, 57000, 230660]
[10, 15000, 205420]
[11, 30800, 3520300]
[12, 431000, 11410900]
[13, 151000, 4031400]
[14, 8000, 183850]
[16, 52500, 5614000]
[17, 120000, 2413600]
[19, 51000, 2973000]
[20, 41559000, 102722660]
[23, 28800, 293020]
[27, 943000, 53940300]
[35, 8000, 15880]
[38, 6000, 24100]
[41, 3000, 47060]
[66, 74500, 2873325]
[69, 2000, 14780]
[81, 1819000, 8266050]
[83, 110000, 1173640]
[87, 30000, 286550]
[101, 297000, 4242000]
[116, 10000, 104000]
[119, 136000, 277080]
[123, 2494000, 26514360]
[135, 2842000, 19092820]
[136, 2040000, 3823680]
[144, 32000, 373320]
[148, 14000, 334750]
[151, 138000, 701530]
[152, 12000, 85585]
[165, 54000, 295660]
[168, 2000, 164500]
[173, 8000, 21880]
[175, 3658000, 35341980]
[177, 8000, 62460]
[179, 5500, 49005]
[182, 10000, 7000]
[189, 145000, 1169880]
[200, 77000, 718760]
[220, 95000, 739150]
[241, 4316000, 24875220]
[242

In [240]:
os.system('open -a /Applications/Microsoft\ Excel.app/ '+DailyFileName);